In [1]:
import requests
import datetime
import pandas as pd
from datetime import timedelta

date = datetime.datetime.now() - timedelta(days=1)
datestr = date.strftime("%Y%m%d")

res = requests.get('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=json&date={}&type=ALLBUT0999'.format(datestr))

In [46]:
df = pd.DataFrame(res.json()['data9'], columns=res.json()['fields9'])
print(len(df))
df.head()

1125


,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0050,元大台灣50,"5,700,896","6,513","770,243,301",135.25,135.65,134.50,135.25,<p style= color:red>+</p>,0.55,135.25,51,135.30,70,0.00
1,0051,元大中型100,"36,956",35,"1,752,458",47.49,47.50,47.22,47.45,<p style= color:red>+</p>,0.13,47.34,27,47.45,1,0.00
2,0052,富邦科技,"920,103",814,"116,518,108",126.80,127.50,125.85,126.85,<p style= color:red>+</p>,1.05,126.75,17,126.85,15,0.00
3,0053,元大電子,"47,216",40,"3,159,293",68.80,68.80,65.65,66.20,<p style= color:red>+</p>,0.60,66.20,1,66.25,1,0.00
4,0054,元大台商50,"21,006",9,"640,952",30.52,30.63,30.49,30.49,<p style= color:green>-</p>,0.05,30.51,53,30.70,3,0.00


In [47]:
up_down_dict = {}
for i in set(df['漲跌(+/-)']):
    if '-' in i:
        up_down_dict[i] = '-'
    elif '+' in i:
        up_down_dict[i] = '+'
    else:
        up_down_dict[i] = 'X'

up_down_dict

{'<p style= color:red>+</p>': '+',
 '<p> </p>': 'X',
 '<p>X</p>': 'X',
 '<p style= color:green>-</p>': '-'}

In [48]:
# 重置漲跌
df['漲跌(+/-)'] = df['漲跌(+/-)'].replace(up_down_dict)
print(len(df))
df.head()

1125


,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0050,元大台灣50,"5,700,896","6,513","770,243,301",135.25,135.65,134.50,135.25,+,0.55,135.25,51,135.30,70,0.00
1,0051,元大中型100,"36,956",35,"1,752,458",47.49,47.50,47.22,47.45,+,0.13,47.34,27,47.45,1,0.00
2,0052,富邦科技,"920,103",814,"116,518,108",126.80,127.50,125.85,126.85,+,1.05,126.75,17,126.85,15,0.00
3,0053,元大電子,"47,216",40,"3,159,293",68.80,68.80,65.65,66.20,+,0.60,66.20,1,66.25,1,0.00
4,0054,元大台商50,"21,006",9,"640,952",30.52,30.63,30.49,30.49,-,0.05,30.51,53,30.70,3,0.00


In [55]:
# 刪除當天沒有開盤價的資料
df = df[df['開盤價'] != '--'].reset_index(drop=True)
print(len(df))
df.head()

1114


,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0050,元大台灣50,"5,700,896","6,513","770,243,301",135.25,135.65,134.50,135.25,+,0.55,135.25,51,135.30,70,0.00
1,0051,元大中型100,"36,956",35,"1,752,458",47.49,47.50,47.22,47.45,+,0.13,47.34,27,47.45,1,0.00
2,0052,富邦科技,"920,103",814,"116,518,108",126.80,127.50,125.85,126.85,+,1.05,126.75,17,126.85,15,0.00
3,0053,元大電子,"47,216",40,"3,159,293",68.80,68.80,65.65,66.20,+,0.60,66.20,1,66.25,1,0.00
4,0054,元大台商50,"21,006",9,"640,952",30.52,30.63,30.49,30.49,-,0.05,30.51,53,30.70,3,0.00


In [56]:
# 將所有df中的元素都變成字串，並將字串中的逗號「,」刪除
df = df.astype(str)
df = df.applymap(lambda s: s.replace(',', ''))

# 將 df 證券代號變成 index
df = df.set_index('證券代號')

# 將 df 中的元素從字串變成數字
# errors='ignore' 代表遇到錯誤跳過，課程中使用 'coerce' 代表遇到錯誤賦予 NaN
df = df.apply(lambda s: pd.to_numeric(s, errors='ignore'))
df.head()

,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
證券代號,,,,,,,,,,,,,,,
0050,元大台灣50,5700896,6513,770243301,135.25,135.65,134.50,135.25,+,0.55,135.25,51,135.30,70,0.0
0051,元大中型100,36956,35,1752458,47.49,47.50,47.22,47.45,+,0.13,47.34,27,47.45,1,0.0
0052,富邦科技,920103,814,116518108,126.80,127.50,125.85,126.85,+,1.05,126.75,17,126.85,15,0.0
0053,元大電子,47216,40,3159293,68.80,68.80,65.65,66.20,+,0.60,66.20,1,66.25,1,0.0
0054,元大台商50,21006,9,640952,30.52,30.63,30.49,30.49,-,0.05,30.51,53,30.70,3,0.0


In [57]:
# 要刪除沒有用的columns
# 其中 axis=1 為是說每條columns去檢查有沒有NaN
# how='all' 是說假如全部都是 NaN 則刪除該 column
# （原本的方法） df = df[df.columns[df.isnull().sum() != len(df)]]

df.dropna(axis=1, how='all', inplace=True)
print(len(df))
df.head()

1114


,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
證券代號,,,,,,,,,,,,,,,
0050,元大台灣50,5700896,6513,770243301,135.25,135.65,134.50,135.25,+,0.55,135.25,51,135.30,70,0.0
0051,元大中型100,36956,35,1752458,47.49,47.50,47.22,47.45,+,0.13,47.34,27,47.45,1,0.0
0052,富邦科技,920103,814,116518108,126.80,127.50,125.85,126.85,+,1.05,126.75,17,126.85,15,0.0
0053,元大電子,47216,40,3159293,68.80,68.80,65.65,66.20,+,0.60,66.20,1,66.25,1,0.0
0054,元大台商50,21006,9,640952,30.52,30.63,30.49,30.49,-,0.05,30.51,53,30.70,3,0.0


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1114 entries, 0050 to 9958
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   證券名稱     1114 non-null   object 
 1   成交股數     1114 non-null   int64  
 2   成交筆數     1114 non-null   int64  
 3   成交金額     1114 non-null   int64  
 4   開盤價      1114 non-null   float64
 5   最高價      1114 non-null   float64
 6   最低價      1114 non-null   float64
 7   收盤價      1114 non-null   float64
 8   漲跌(+/-)  1114 non-null   object 
 9   漲跌價差     1114 non-null   float64
 10  最後揭示買價   1114 non-null   object 
 11  最後揭示買量   1114 non-null   int64  
 12  最後揭示賣價   1114 non-null   object 
 13  最後揭示賣量   1114 non-null   int64  
 14  本益比      1114 non-null   float64
dtypes: float64(6), int64(5), object(4)
memory usage: 139.2+ KB


In [59]:
df[df['收盤價']/df['開盤價'] > 1.05]

,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
證券代號,,,,,,,,,,,,,,,
2331,精英,85891844,30674,2933490058,32.35,35.50,31.95,34.5,+,2.20,34.50,316,34.55,12,90.79
2338,光罩,101380210,36094,5410510525,51.60,54.20,50.30,54.2,+,4.85,54.00,2,54.20,473,35.89
2465,麗臺,8235693,4244,255494756,29.50,32.85,28.80,32.8,+,2.90,32.75,4,32.80,58,0.00
2477,美隆電,8463705,3514,298881127,32.65,35.50,32.55,35.5,+,3.20,35.50,126,--,0,25.18
2727,王品,5672459,4597,1012092665,171.50,183.00,169.00,180.5,+,10.00,180.00,57,180.50,2,39.41
3545,敦泰,73160232,41135,8686023140,115.50,123.00,114.00,123.0,+,11.00,122.50,16,123.00,1903,50.83
4571,鈞興-KY,736503,614,131911177,174.50,184.50,174.50,184.5,+,11.50,184.50,6,185.00,13,37.58
6243,迅杰,4565772,1928,81969049,17.20,18.75,16.90,18.4,+,0.90,18.30,3,18.40,60,0.00
8482,商億-KY,388001,239,32839889,82.00,89.20,81.80,89.2,+,8.10,89.20,22,--,0,14.50
